Imports

In [ ]:
# lib reloading
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# python standard packages
import logging 
from re import compile
from time import time, sleep

# third-party packages
import keyring

from selenium.webdriver.support.select import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

# in project packages
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))  # add parent folder to paths
from utilities.chrome_driver import ChromeDriver

# logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# project constants
ELEMENT_TIMEOUT = 10  # [s]
WAIT_FOR_ELEMENT = 0.5  # [s]

# needed to be setup earlier
# keyring.set_password("credential_name", "user_name", "password")
UNITY_CREDENTIALS = ("unity_id_cred", "mateusz.gosciniak.dev@gmail.com")

Methods

In [ ]:
def wait_for_element(chrome_driver, by: str = By.ID, value: str | None = None) -> WebElement:
    """ Wrapper on selenium method find_element.
    Added wait time checking if element appears. 
    Find an element given a By strategy and locator.

    Constants needed to be setup:
    ELEMENT_TIMEOUT = 15
    WAIT_FOR_ELEMENT = 0.5

    :Usage:
        element = wait_for_element(driver, By.ID, 'foo')

    :rtype: WebElement
    """

    start_time = time()
    web_element = None
    while not web_element:
        try:
            web_element = chrome_driver.find_element(by, value)
        except NoSuchElementException as ex:
            if time() - start_time < ELEMENT_TIMEOUT:
                sleep(WAIT_FOR_ELEMENT)
                continue
            
            logging.error(f"element not found: {ex}")
            raise ex
    return web_element

def authenticate_asset_store(chrome_driver, selenium_waiter):
    # open main page
    chrome_driver.get("https://assetstore.unity.com/")
    logging.info("Asset Store open")

    # accept cookies
    accept_all_cookies_btn = chrome_driver.find_element(By.ID, "onetrust-accept-btn-handler")
    selenium_waiter.until(EC.element_to_be_clickable(accept_all_cookies_btn))
    accept_all_cookies_btn.click()
    logging.info("Cookies accepted")

    # open login page
    chrome_driver.get("https://assetstore.unity.com/auth/login?redirect_to=%2F")
    login_page = chrome_driver.current_url
    logging.info("Login Page open")

    # write email
    email_field = chrome_driver.find_element(By.ID, "conversations_create_session_form_email")
    selenium_waiter.until(EC.element_to_be_clickable(email_field))
    email_field.send_keys(UNITY_CREDENTIALS[1])
    logging.info("Email filled")

    # write password
    password_field = chrome_driver.find_element(By.ID, "conversations_create_session_form_password")
    selenium_waiter.until(EC.element_to_be_clickable(password_field))
    password_field.send_keys(keyring.get_password(*UNITY_CREDENTIALS))
    logging.info("Password filled")

    # click submit
    submit_btn = chrome_driver.find_element(By.XPATH, "//input[@type='submit']")
    selenium_waiter.until(EC.element_to_be_clickable(submit_btn))
    submit_btn.click()
    logging.info("Form Submited")

    # check if proper page and credentials are ok
    page = chrome_driver.page_source
    unity_asset_store_desc = "Unity Asset Store - The Best Assets"
    if not unity_asset_store_desc in page:
        if chrome_driver.current_url == login_page:
            raise Exception("Bad Credentials")
        else:
            raise Exception("Wrong Page obtained after login, maybe 2FA enabled?")
    logging.info("Asset Store authenticated")

def get_assets_from_page(chrome_driver, selenium_waiter):
    # go to asset store by adress url with query url
    chrome_driver.get("https://assetstore.unity.com/?free=true&exclude=true&orderBy=1")
    logging.info("Asset Store filtered by free to download")

    # Get how many results is on page
    results_div = chrome_driver.find_element(By.XPATH, "//div[text()[contains(.,'results')]]")
    results_text = results_div.text
    # Pattern to find 3 groups of numbers in string ex. 1-24 of 104 results
    regex_pattern = compile(r'(\d+)-(\d+) of (\d+)')
    regex_match = regex_pattern.match(results_text)
    if not regex_match:
        raise Exception("No assets to add")

    if len(regex_match.groups()) != 3:
        raise Exception("No assets to add")

    # Geting last number as total amount of results
    avaiable_assets = int(regex_match.group(3))

    # Find table of assets
    asset_grid = chrome_driver.find_element(By.XPATH, "//div[@data-test='asset-grid']/div")
    selenium_waiter.until(EC.element_to_be_clickable(asset_grid))
    if not "Add to My Assets" in asset_grid.text:
        raise Exception("No assets to add")

    # Get assets as collections
    assets = asset_grid.find_elements(By.XPATH, "./div")
    if len(assets) == 0:
        raise Exception("No assets to add")

    logging.info(f"Assets count: {avaiable_assets}, on this page is {len(assets)}")
    for asset in assets:
        logging.info(f"Asset to add: {','.join(asset.text.split('\n'))}")
        _, asset_btn_add = asset.find_elements(By.XPATH, ".//button[*]")
        
        # if Open in Unity button appear
        if asset_btn_add.text != "Add to My Assets":
            if asset_btn_add.text == "Request access":
                return True # end of processing
            continue

        asset_btn_add.click()
        logging.info(f"Add asset clicked")

        accept_btn = wait_for_element(chrome_driver, By.XPATH, "//button[@label='Accept']")
        selenium_waiter.until(EC.element_to_be_clickable(accept_btn))
        accept_btn.click()
        logging.info(f"Accept button clicked")
        
        # Added to My Assets Popup check 
        added_popup = wait_for_element(chrome_driver, By.XPATH, "//div[text()[contains(.,'Added to My Assets')]]")
        selenium_waiter.until(EC.element_to_be_clickable(added_popup))
        logging.info(f"Added to My Assets Popup appear")
        
        # close popup by sending Escape key
        sleep(1)
        ActionChains(chrome_driver).send_keys(Keys.ESCAPE).perform()
        logging.info(f"Send ESC to close popup")
        
    logging.info(f"Page finished - refreshing")
    return False


Chrome Driver Initialization

In [ ]:
chrome_driver = ChromeDriver().chrome_driver
selenium_waiter = WebDriverWait(chrome_driver, timeout=ELEMENT_TIMEOUT)
logging.info("Selenium Inited")

Asset Store Authentication

In [ ]:
authenticate_asset_store(chrome_driver, selenium_waiter)

Asset Store grab all assets from page 

In [ ]:
while not get_assets_from_page(chrome_driver, selenium_waiter):
    logging.info(f"Found more assets - page refreshing")

Close Chrome

In [ ]:
chrome_driver.quit()